<a href="https://colab.research.google.com/github/chunyuan0221/3rd-ML100days/blob/master/Day_93_CNN_Brief_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業目標:
    運用 Keras 模組建構CNN, 了解 CNN 的架構
    
    

# 作業重點
(1)嘗試比對 Dense 與 layers.Conv2D 架構NN 的差異

(2) 有沒有Pooling layer, 對於參數量的差異

注意: input_shape 請勿修改

In [1]:
#導入相關模組
import keras
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
#確認keras 版本
print(keras.__version__)

Using TensorFlow backend.


2.2.5


In [0]:
input_shape = (28, 28, 1)

### 建立一般的神經網路(NN)
***
- 三種方法

In [3]:
input_layer = keras.layers.Input(shape=input_shape)
x = Dense(units=100, activation='relu')(input_layer)
out = Dense(units=10, activation='softmax')(x)
NN_model = keras.models.Model(inputs=[input_layer], outputs=[out])
NN_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
dense_1 (Dense)              (None, 28, 28, 100)       200       
_________________________________________________________________
dense_2 (Dense)              (None, 28, 28, 10)        1010      
Total params: 1,210
Trainable params: 1,210
Non-trainable params: 0
_________________________________________________________________


In [4]:
NN_model = Sequential()
NN_model.add(Dense(units=100, activation='relu', input_shape=(28, 28, 1)))
NN_model.add(Dense(units=10, activation='softmax'))
NN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 28, 28, 100)       200       
_________________________________________________________________
dense_4 (Dense)              (None, 28, 28, 10)        1010      
Total params: 1,210
Trainable params: 1,210
Non-trainable params: 0
_________________________________________________________________


In [5]:
model = Sequential([Dense(units=100, input_shape=input_shape),
                    Activation('relu'),
                    Dense(units=10),
                    Activation('softmax')])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 28, 28, 100)       200       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 100)       0         
_________________________________________________________________
dense_6 (Dense)              (None, 28, 28, 10)        1010      
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 10)        0         
Total params: 1,210
Trainable params: 1,210
Non-trainable params: 0
_________________________________________________________________


# layers.Conv2D 模型, 用作比對

![CNN_Model.png](attachment:CNN_Model.png)



In [6]:
#建立一個序列模型
model = models.Sequential()

#建立兩個卷績層, 32 個內核, 內核大小 3x3, 
#輸入影像大小 28x28x1
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Conv2D(32, (3, 3)))

#建立平坦層，打成1維
model.add(Flatten())

#建立一個全連接層
model.add(Dense(units=100))
model.add(Activation('relu'))

#建立一個輸出層, 並採用softmax
model.add(Dense(units=10))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               1843300   
_________________________________________________________________
activation_3 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1010      
_________________________________________________________________
activation_4 (Activation)    (None, 10)               

### 加入Pooling比較參數量


In [7]:
#建立一個序列模型
model = models.Sequential()

#建立兩個卷績層, 32 個內核, 內核大小 3x3, 
#輸入影像大小 28x28x1
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

#建立平坦層，打成1維
model.add(Flatten())

#建立一個全連接層
model.add(Dense(units=100))
model.add(Activation('relu'))

#建立一個輸出層, 並採用softmax
model.add(Dense(units=10))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               80100     
_________________________________________________________________
activation_5 (Activation)    (None, 100)              

### Result
1. 嘗試比對 Dense 與 layers.Conv2D 架構NN 的差異

2. 有沒有Pooling layer, 對於參數量的差異
***
1. 加入Conv2D的參數量比沒加入的多了不少參數，運算如下:
  - 前言 : 參數量指的是權重(weights)+偏移值(bias)
  - Conv2D : input_image會與filter進行converlution，在共享權重下:
          初始input_image shape = (28, 28, 1)
          weights = (3*3) * 32 = 288
          bias = 32
          total params = 288 + 32 = 320

          做完第一次Conv2D後，image shape = (26, 26, 32)
          weights = image_depth * filter_size * filter_number
                  = 32 * (3*3) * 32 = 9216
          bias = filter_number = 32
          total params = 9216 + 32 = 9248

2. 當Pooling_size=(2x2)，Pooling layer做的事情只是從這(2x2)框框內取一個最大或平均值出來，因此：
  - 在Pooling layer並不會產生參數，但會改變圖片大小(image_shape)
  - 因為做了Pooling讓圖片變小，對之後進行扁平層時減少了大量資料，因此在後面做權連接層上卻可以減少不少參數。
          我由上面的程式來看:
          1. 在2次Conv2D後 : image_shape = (24, 24, 32)
          2. 進行Flatten後 : image_shape = (None, 18432)
          我們在跳掉加入Pooling的情況下來看:
          1. 在第2次Pooling後 : image_shape = (5, 5, 32)
          2. 進行Flatten後 : image_shape = (None, 800)
          **這時候我們可以看到，由於做了Pooling讓圖片變小，我們將圖片轉成1維後的資料量變少，相對的在後面的訓練上可以減少不少params(weights+bias)**


